我们使用pycaret.nlp模块进行自然语言处理。

学习内容：

    设置环境：在PyCaret中设置环境并执行关键的文本预处理任务。
    创建模型：创建topic model。
    分配模型：使用经过训练的模型将文档/文本分配给主题。
    绘制图表：使用图表分析topic model/语料库。

# 1.数据集处理
数据集包含6818个样本。特征的描述如下：

    country	：借款国
    en：申请贷款时借款人的个人经历
    gender：性别（M =男性，F =女性）
    loan_amount：已批准和已支付的贷款金额
    nonpayment：贷方类型（lender	：Kiva网站上的个人注册用户，partner：与Kiva合作寻找并资助贷款的小额信贷机构）
    sector：借款人行业
    status：贷款状态（1：未还款，0：还款）

In [13]:
from pycaret.datasets import get_data
import pandas as pd


data = pd.read_csv('./data/LDA_data.csv')
data.head()

,country,en,gender,loan_amount,nonpayment,sector,status
0,Dominican Republic,"""Banco Esperanza"" is a group of 10 women looki...",F,1225,partner,Retail,0
1,Dominican Republic,"""Caminemos Hacia Adelante"" or ""Walking Forward...",F,1975,lender,Clothing,0
2,Dominican Republic,"""Creciendo Por La Union"" is a group of 10 peop...",F,2175,partner,Clothing,0
3,Dominican Republic,"""Cristo Vive"" (""Christ lives"" is a group of 10...",F,1425,partner,Clothing,0
4,Dominican Republic,"""Cristo Vive"" is a large group of 35 people, 2...",F,4025,partner,Food,0


数据集共有6818条，若电脑无法运行这么大的话，取其中1000条运行。（后续绘图会比较卡）

In [14]:
#check the shape of data
data.shape

(6818, 7)

卡就运行，不卡就别运行了。

In [15]:
# sampling the data to select only 1000 documents
data = data.sample(1000, random_state=786).reset_index(drop=True)
data.shape

(1000, 7)

# 2. 设置环境
setup()函数会初始化pycaret中的环境，并执行一些必须处理NLP问题的文本预处理步骤。在pycaret中执行任何其他功能之前，必须先调用setup()函数。

执行设置后，将自动应用以下预处理步骤：

    - 删除数字字符：从文本中删除所有数字字符。
    - 删除特殊字符：从文本中删除所有非字母数字的特殊字符。
    - Word Tokenization：单词标记化是将大量文本样本拆分为多个单词的过程。
    - 停用词的删除：很常见，即使对于语言有意义，它也对信息检索几乎没有价值。“the”，“a”，“an”，“in”
    还有很多。。。。。。
    

安装需要的语言模型
python命令行界面，例如Anaconda Prompt，在命令行中输入以下内容：
- python -m spacy download en_core_web_sm
- python -m textblob.download_corpora

然后运行下面的代码

In [22]:
from pycaret.nlp import *
exp_nlp101 = setup(data = data, target = 'en', session_id = 123)

INFO:logs:PyCaret NLP Module
INFO:logs:version 2.3.5
INFO:logs:Initializing setup()
INFO:logs:USI: cfce
INFO:logs:setup(data=(1000, 7), target=en, custom_stopwords=None, html=True, session_id=123, log_experiment=False,
                    experiment_name=None, log_plots=False, log_data=False, verbose=True)
INFO:logs:Checking environment
INFO:logs:python_version: 3.8.12
INFO:logs:python_build: ('default', 'Oct 12 2021 06:23:56')
INFO:logs:machine: x86_64
INFO:logs:platform: macOS-10.16-x86_64-i386-64bit
INFO:logs:Memory: svmem(total=17179869184, available=6865436672, percent=60.0, used=9613197312, free=298565632, active=6486667264, inactive=6497939456, wired=3126530048)
INFO:logs:Physical Core: 4
INFO:logs:Logical Core: 8
INFO:logs:Checking libraries
INFO:logs:pd==1.3.4
INFO:logs:numpy==1.21.2
INFO:logs:gensim==3.8.3
INFO:logs:spacy==2.3.7
INFO:logs:nltk==3.6.5
INFO:logs:textblob==0.17.1
INFO:logs:pyLDAvis==3.2.2
INFO:logs:wordcloud==1.8.1
INFO:logs:mlflow==1.21.0
INFO:logs:Checking Exc

Traceback (most recent call last):
  File "/Users/hanyan/opt/anaconda3/envs/python38/lib/python3.8/site-packages/pycaret/nlp.py", line 313, in setup
    sp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
  File "/Users/hanyan/opt/anaconda3/envs/python38/lib/python3.8/site-packages/spacy/__init__.py", line 30, in load
    def load(
  File "/Users/hanyan/opt/anaconda3/envs/python38/lib/python3.8/site-packages/spacy/util.py", line 163, in load_model
    return reg.find(legacy_name)
OSError: [E049] Can't find spaCy data directory: 'None'. Check your installation and permissions, or use spacy.util.set_data_path to customise the location if necessary.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/hanyan/opt/anaconda3/envs/python38/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/2c/971vnt113tb76xbh7

TypeError: object of type 'NoneType' has no len()

当执行setup()时，所有文本预处理步骤都会自动执行,这些步骤对于在NLP中都是必不可少的。这里极大地简化了这个过程。

# 3.创建模型
在PyCaret中创建topic model非常简单，使用create_model()函数创建topic model.该函数采用一个强制性参数，即模型名称作为字符串。此函数返回一个训练好的模型对象。

PyCaret中有5种topic model。这里我们使用LDA模型：

In [7]:
lda = create_model('lda')

INFO:logs:Initializing create_model()
INFO:logs:create_model(model=lda, multi_core=False, num_topics=None, verbose=True, system=True)
INFO:logs:Checking exceptions
INFO:logs:Preloading libraries
INFO:logs:Preparing display monitor


NameError: name 'html_param' is not defined

In [8]:
print(lda)

NameError: name 'lda' is not defined

# 4. 分配模型
现在我们已经创建了一个topic model，我们将给数据集（6818个文档/样本）倒入模型加以分析结果。 我们将使用assign_model()函数来实现。

In [ ]:
lda_results = assign_model(lda)
lda_results.head()

现在dataframe里增加了6个列。Topic_0 ... Topic_3是主题比例（默认分为4类，可以修改），代表每个文档的主题分布。Dominant_Topic是具有最高比例的topic编号，而Perc_Dominant_Topic是占主导topic的比例。

# 4. 绘制图表
plot_model()函数可用于分析整个语料库或仅分析通过主题模型提取的特定主题。 

### 整个语料库的单词频率分布

In [ ]:
plot_model()

### 9.3 Topic 1的频率分布
plot_model()也可以用于分析特定topic。

In [ ]:
plot_model(lda, plot = 'frequency', topic_num = 'Topic 1')

### topic的分布情况

In [ ]:
plot_model(lda, plot = 'topic_distribution')

### t-SNE
T分布随机邻居嵌入（t-SNE）是一种非线性降维技术，非常适合在二维或三维低维空间中嵌入高维数据以进行可视化。

In [ ]:
plot_model(lda, plot = 'tsne')